In [ ]:
import os
import numpy as np
import time
import matplotlib.pyplot as plt
import pandas as pd
import math
import pickle


def prepare_market_data():
    
    sectors_info = pd.read_csv("sectors.csv")
    tickers = sectors_info["TICKER"].values
    dict_close_px = {}
    dict_volumes = {}
    
    for ticker in tickers:
        try:
            market_data = pd.read_csv(os.path.join(os.path.abspath("market_data"),\
                                                   ticker+".csv"), index_col = 0)
            market_data = market_data.reindex(pd.to_datetime(market_data.index))
            close_px_daily = market_data["close"].resample("1D").last()
            volume_daily = market_data["volume"].resample("1D").sum()
            dict_close_px[ticker] = close_px_daily
            dict_volumes[ticker] = volume_daily
            
        except Exception as e:
            print(e)
            
    return dict_close_px, dict_volumes
    

dict_close_px, dict_volumes = prepare_market_data()
common_index = dict_close_px[list(dict_close_px.keys())[0]].index

for key in dict_close_px.keys():
    common_index = common_index.union(dict_close_px[key].index)

for key in dict_volumes.keys():
    common_index = common_index.union(dict_volumes[key].index)
    
daily_px_df = pd.DataFrame(index = common_index)

for key in dict_close_px.keys():
    daily_px_df[key] = dict_close_px[key]
    
daily_volume_df = pd.DataFrame(index = common_index)

for key in dict_volumes.keys():
    daily_volume_df[key] = dict_volumes[key]
    
daily_px_df.to_csv("market_data_daily_resample_close.csv")  
daily_volume_df.to_csv("market_data_daily_resample_volume.csv")  